<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 10:20:45
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.41 C
-------------------
Today PnL: 75.72 K (0.54%)
Current PnL: -17.55 L (-11.8%)
CY Booked + Current PnL: -7.69 L (-5.17%)
-------------------
Total profit:  2.98 L
Total loss:  -20.53 L
-------------------
Total Booked + Current PnL: 19.29 L (15.85%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.02%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.26 L (55.7%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-28.18,72.0,H-MC,7.99,143680.0,28744.0,7845.0,0.59,25.01,5.46,31.83,81.0,3.66,1.08,44.87,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.19,58.0,H-LC,10.47,171380.0,15203.0,10883.0,-0.19,9.73,6.35,16.71,22.0,1.40,1.29,28.87,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,105.76,65.0,M-SC,6.18,88694.0,-12083.0,12178.0,-0.38,-11.99,13.73,0.09,245.0,-0.99,0.67,16.52,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,50.0,M-LC,4.53,214902.0,13422.0,13797.0,0.54,6.66,6.42,13.51,60.0,0.97,1.61,20.10,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192378.0,41465.0,15833.0,2.49,27.48,8.23,37.97,93.0,2.62,1.44,59.73,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.68,36.0,H-MC,2.15,134820.0,-968.0,43722.0,-0.03,-0.71,32.43,31.48,107.0,-0.02,1.01,11.32,X40,ATH,INSURANCE
31,HAVELLS,2069.16,0.20,62.0,H-MC,1.89,248508.0,777.0,74279.0,0.66,0.31,29.89,30.30,92.0,0.01,1.87,13.65,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.58,41.0,H-MC,3.95,136530.0,397.0,30733.0,0.25,0.29,22.51,22.86,91.0,0.01,1.02,12.94,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.41,55.0,H-MC,4.87,201240.0,420.0,68361.0,-0.58,0.21,33.97,34.25,80.0,0.01,1.51,5.42,X40,ATH,FMCG
88,WIPRO,420.00,-13.79,41.0,M-LC,6.87,151218.0,273.0,109179.0,0.09,0.18,72.20,72.51,53.0,0.00,1.14,6.21,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.37,32.0,H-LC,9.98,222783.0,-19141.0,82407.0,-1.68,-7.91,36.99,26.15,8.0,-0.23,1.67,5.28,X40,ATH,IT
41,INFY,2275.00,-21.39,38.0,H-LC,10.87,306294.0,-6594.0,178294.0,-0.46,-2.11,58.21,54.87,3.0,-0.04,2.30,4.40,X40,BTT,IT
85,VBL,671.64,-15.83,39.0,H-LC,6.90,302890.0,-12952.0,125608.0,1.09,-4.10,41.47,35.67,5.0,-0.10,2.27,9.19,X40N,ATH,FMCG
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286540.0,-59416.0,183271.0,-1.53,-17.17,63.96,35.80,1.0,-0.32,2.15,1.51,X40,BTT,IT
45,JIOFIN,387.00,-1.31,43.0,H-LC,13.98,218295.0,889.0,52609.0,0.79,0.41,24.10,24.61,48.0,0.02,1.64,55.45,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.36,49.0,H-LC,1.71,232380.0,-13771.0,92975.0,-0.61,-5.59,40.01,32.18,16.0,-0.15,1.74,28.43,X200,ATH,IT
52,NESTLEIND,1377.00,-8.60,60.0,H-LC,2.74,281057.0,15631.0,41147.0,-0.61,5.89,14.64,21.40,11.0,0.38,2.11,13.40,XY25,NTT,FMCG
43,ITC,452.00,-37.92,49.0,H-LC,3.12,199805.0,-333.0,21219.0,0.31,-0.17,10.62,10.44,4.0,-0.02,1.50,5.39,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,52.0,H-LC,5.79,214062.0,9562.0,31852.0,0.59,4.68,14.88,20.25,10.0,0.30,1.61,13.82,X40N,BTT,PHARMA
85,VBL,671.64,-15.83,39.0,H-LC,6.90,302890.0,-12952.0,125608.0,1.09,-4.10,41.47,35.67,5.0,-0.10,2.27,9.19,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.0,1112.12,80.0,M-SC,3.21,164934.0,-16209.0,61669.0,8.45,-8.95,37.39,25.10,235.0,-0.26,1.24,32.18,XY24,NTT,HEALTHCARE
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
69,SONACOMS,1006.0,-35.63,54.0,M-SC,12.46,80168.0,-20991.0,95881.0,3.85,-20.75,119.60,74.03,202.0,-0.22,0.60,13.04,AR,BTT,AUTO
0,5PAISA,593.0,130.77,56.0,H-SC,8.83,124373.0,-62559.0,71912.0,3.64,-33.47,57.82,5.00,153.0,-0.87,0.93,18.98,OX40N,NTT,FINANCE
55,RAJESHEXPO,518.0,1713.13,54.0,L-SC,8.39,49476.0,-87701.0,87795.0,3.26,-63.93,177.45,0.07,267.0,-1.00,0.37,23.05,OX40N,NTT,JEWELLERY


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.37,32.0,H-LC,9.98,222783.0,-19141.0,82407.0,-1.68,-7.91,36.99,26.15,8.0,-0.23,1.67,5.28,X40,ATH,IT
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286540.0,-59416.0,183271.0,-1.53,-17.17,63.96,35.80,1.0,-0.32,2.15,1.51,X40,BTT,IT
14,BLUESTARCO,2326.38,11.01,58.0,H-MC,3.36,191550.0,26880.0,41087.0,-1.44,16.32,21.45,41.28,89.0,0.65,1.44,25.61,X40N,ATH,AC
39,INDIGOPNTS,1408.00,114.29,49.0,M-SC,3.55,140480.0,-34079.0,34109.0,-1.09,-19.52,24.28,0.02,221.0,-1.00,1.05,22.19,OX40N,NTT,PAINTS
26,DMART,5201.00,-5.97,69.0,H-LC,15.49,234872.0,50391.0,19988.0,-0.75,27.31,8.51,38.14,38.0,2.52,1.76,40.82,X40N,NTT,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,105.76,65.0,M-SC,6.18,88694.0,-12083.0,12178.0,-0.38,-11.99,13.73,0.09,245.0,-0.99,0.67,16.52,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,85.53,48.0,H-SC,12.91,117207.0,-25272.0,44797.0,1.97,-17.74,38.22,13.70,163.0,-0.56,0.88,38.34,OX40N,NTT,BANKS
46,KANSAINER,340.0,-68.34,53.0,H-SC,2.90,218151.0,-51516.0,87849.0,-0.20,-19.10,40.27,13.47,138.0,-0.59,1.64,10.34,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.55,140480.0,-34079.0,34109.0,-1.09,-19.52,24.28,0.02,221.0,-1.00,1.05,22.19,OX40N,NTT,PAINTS
18,CERA,9475.0,-19.57,47.0,H-SC,2.53,114606.0,-29197.0,55939.0,0.41,-20.30,48.81,18.60,149.0,-0.52,0.86,26.47,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286540.0,-59416.0,183271.0,-1.53,-17.17,63.96,35.80,1.0,-0.32,2.15,1.51,X40,BTT,IT
41,INFY,2275.00,-21.39,38.0,H-LC,10.87,306294.0,-6594.0,178294.0,-0.46,-2.11,58.21,54.87,3.0,-0.04,2.30,4.40,X40,BTT,IT
43,ITC,452.00,-37.92,49.0,H-LC,3.12,199805.0,-333.0,21219.0,0.31,-0.17,10.62,10.44,4.0,-0.02,1.50,5.39,X40,NTT,FMCG
85,VBL,671.64,-15.83,39.0,H-LC,6.90,302890.0,-12952.0,125608.0,1.09,-4.10,41.47,35.67,5.0,-0.10,2.27,9.19,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,12.88,248674.0,-12945.0,132170.0,1.24,-4.95,53.15,45.57,7.0,-0.10,1.87,5.81,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192378.0,41465.0,15833.0,2.49,27.48,8.23,37.97,93.0,2.62,1.44,59.73,AR,ATH,AUTO
25,DIXON,18940.15,-28.18,72.0,H-MC,7.99,143680.0,28744.0,7845.0,0.59,25.01,5.46,31.83,81.0,3.66,1.08,44.87,X40N,ATH,IT
26,DMART,5201.00,-5.97,69.0,H-LC,15.49,234872.0,50391.0,19988.0,-0.75,27.31,8.51,38.14,38.0,2.52,1.76,40.82,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,11.01,58.0,H-MC,3.36,191550.0,26880.0,41087.0,-1.44,16.32,21.45,41.28,89.0,0.65,1.44,25.61,X40N,ATH,AC
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192378.0,41465.0,15833.0,2.49,27.48,8.23,37.97,93.0,2.62,1.44,59.73,AR,ATH,AUTO
86,VOLTAS,1918.49,2.14,63.0,H-MC,3.68,214230.0,22488.0,73545.0,-0.32,11.73,34.33,50.08,99.0,0.31,1.61,19.32,XY25,ATH,AC
31,HAVELLS,2069.16,0.20,62.0,H-MC,1.89,248508.0,777.0,74279.0,0.66,0.31,29.89,30.30,92.0,0.01,1.87,13.65,X40,ATH,ELECTRICAL
23,DABUR,735.00,-1.13,65.0,H-MC,4.86,215046.0,18018.0,71610.0,0.86,9.14,33.30,45.49,102.0,0.25,1.61,23.38,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192378.0,41465.0,15833.0,2.49,27.48,8.23,37.97,93.0,2.62,1.44,59.73,AR,ATH,AUTO
51,NATIONALUM,247.44,-42.83,75.0,H-MC,1.27,113272.0,12768.0,17874.0,0.75,12.70,15.78,30.49,79.0,0.71,0.85,49.22,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-38.90,64.0,M-SC,3.62,103275.0,11776.0,66974.0,1.47,12.87,64.85,86.07,223.0,0.18,0.78,49.97,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,50.0,H-SC,12.11,136740.0,13404.0,120359.0,0.97,10.87,88.02,108.45,119.0,0.11,1.03,35.65,AR,ATH,MISC
88,WIPRO,420.00,-13.79,41.0,M-LC,6.87,151218.0,273.0,109179.0,0.09,0.18,72.20,72.51,53.0,0.00,1.14,6.21,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83508.0,-18702.0,112093.0,1.01,-18.30,134.23,91.37,208.0,-0.17,0.63,35.56,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,38.08,86208.0,-27341.0,67389.0,0.76,-24.08,78.17,35.27,268.0,-0.41,0.65,111.00,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.00,46.0,H-SC,15.39,89350.0,-11613.0,106148.0,0.21,-11.50,118.80,93.63,148.0,-0.11,0.67,33.08,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.34,55.0,H-SC,12.90,94170.0,-3810.0,25416.0,1.01,-3.89,26.99,22.05,152.0,-0.15,0.71,36.88,XR,ATH,FINANCE
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,38.08,86208.0,-27341.0,67389.0,0.76,-24.08,78.17,35.27,268.0,-0.41,0.65,111.00,XR,NTT,HOTELS
7,ATULAUTO,844.00,3964.86,78.0,M-SC,8.59,126744.0,-20624.0,72434.0,-0.32,-13.99,57.15,35.16,236.0,-0.28,0.95,29.46,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,3.66,81.0,H-MC,22.05,192378.0,41465.0,15833.0,2.49,27.48,8.23,37.97,93.0,2.62,1.44,59.73,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.36,83508.0,-18702.0,112093.0,1.01,-18.30,134.23,91.37,208.0,-0.17,0.63,35.56,XY25,NTT,FINANCE
66,SHALBY,327.00,1112.12,80.0,M-SC,3.21,164934.0,-16209.0,61669.0,8.45,-8.95,37.39,25.10,235.0,-0.26,1.24,32.18,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,43.52
2,50,73.81


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.22
LC    36.00
MC    25.77
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.04
X40      15.49
X40N     14.24
XY25     10.07
AR        9.54
XR        9.10
OX40N     7.45
MH        1.78
X200      1.74
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.68
H-MC    22.77
H-SC    22.69
M-SC    13.95
M-LC     7.23
M-MC     2.68
L-SC     1.58
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.80,-0.21,34.60
IT,13.31,-16.36,74.06
FINANCE,7.46,-21.39,66.15
BANKS,7.39,-15.79,64.95
PAINTS,6.01,-15.22,41.01
MISC,5.89,-17.43,84.75
ELECTRICAL,5.72,-8.17,67.79
AUTO,4.93,-2.74,47.44
AC,3.78,3.71,28.90


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2730368.0,22
AR,1208041.0,10
XR,1081687.0,13
X40,901979.0,10
X40N,536089.0,11
OX40N,464826.0,10
XY25,409448.0,7
SR,196574.0,2
X5K,125878.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2751874.0,27
H-LC,1431438.0,16
M-SC,1388757.0,17
H-MC,1168320.0,17
M-LC,416158.0,5
M-MC,320551.0,2
L-SC,251087.0,3
L-MC,59589.0,1
L-LC,38631.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925800.0,6
M-SC,XY24,808085.0,7
H-SC,AR,659360.0,5
H-LC,X40,613633.0,5
H-SC,XR,516751.0,6
H-MC,XY24,456019.0,4
H-LC,AR,374420.0,2
M-MC,XY24,320551.0,2
H-LC,X40N,231912.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
